In [1]:
import os
import sys
# Chemin du dossier src
chemin_src = os.path.join('.', 'src')
sys.path.append(chemin_src)

In [2]:
from functions import write_dataframe_to_excel
from functions import read_and_check_input, Separate_data, nettoyer_dataframe
from functions import construire_tableau, Transpose_dataframe
from functions import format_constraints_elements, format_constraints_qualite, format_constraints_MP
from functions import solve_linear_program
from functions import construct_result_dataframe,Save_errors

In [3]:
def create_optimal_recipe(chemin_fichier):
    # Lire les fichiers Excel
    df_table, df_MP, df_contraints, erreurs = read_and_check_input(chemin_fichier)
    if len(erreurs) != 0 :
        return 0
    df_contraints_element, df_contraints_qualite, df_MP_dispo, df_MP_indispo = Separate_data(df_table, df_MP, df_contraints)

    # Suppression des matières premières indisponibles
    df_table = nettoyer_dataframe(df_table, df_MP_indispo)

    # Construction de la matrice A et du vecteur C
    A, C = construire_tableau(df_table, df_MP_dispo)

    # Initialisation des listes pour les contraintes
    constraints = {'A_eq': {},'b_eq': {},'A_sup': {},'b_sup': {} }

    df_contraints_element = Transpose_dataframe(df_contraints_element)
    # Mettre  les contraintes concernant les éléments
    constraints = format_constraints_elements(df_contraints_element, A,constraints)

    # Mettre  les contraintes concernant la qualité
    constraints = format_constraints_qualite(df_contraints_qualite, A,constraints)

    # Mettre les contraintes concernant les matières premières disponibles
    constraints, bounds = format_constraints_MP(df_MP_dispo, constraints)

    # Résoudre le problème d'optimisation linéaire
    method = 'simplex' #'interior-point' 'simplex'
    res, ce_result, ci_result = solve_linear_program(C, constraints,bounds,method)
    conforme = all(val == 1 for val in ce_result) and  all(val == 1 for val in ci_result)    
    # print(method, '\n')
    # print(ce_result, ci_result)

    erreurs = []
    if not ce_result[0] :
        message = "Erreur : La proportion total n'est pas égale à 1"
        erreurs.append(message)

    # On recupere le chemin du dossier data
    dossier_data = os.path.dirname(chemin_fichier)
    if conforme :
        # Construire le DataFrame résultats
        df_res = construct_result_dataframe(df_MP_dispo, df_table, res)
        # Écrire le DataFrame résultats dans le fichier Excel
        write_dataframe_to_excel(df_res, dossier_data, new_sheet_name='Recette')
        print("Le problème admet une solution optimale.")
    else : 
        # Sauvegarder les erreurs dans un fichier texte
        message = "Veillez revoir les contraintes de la feuille 'Contraintes Qualités et Elément' et/ou 'Contraintes matières premières'"
        erreurs.append(message)
        Save_errors(erreurs, dossier_data, res, A, df_contraints_qualite, df_contraints_element, df_MP_dispo )
        print("Les erreurs ont été enregistrées dans le fichier erreurs.txt")
    return 


In [4]:
if __name__ == "__main__":
    # Chemin du fichier
    chemin_fichier = os.path.join('.', 'data', 'recipe_optimization_data.xlsm')
    create_optimal_recipe(chemin_fichier)

Les erreurs ont été enregistrées dans le fichier erreurs.txt


#### Essais


In [1]:
import os
import sys
# Chemin du dossier src
chemin_src = os.path.join('.', 'src')
sys.path.append(chemin_src)

from functions import write_dataframe_to_excel
from functions import read_and_check_input, Separate_data, nettoyer_dataframe
from functions import construire_tableau, Transpose_dataframe
from functions import format_constraints_elements, format_constraints_qualite, format_constraints_MP
from functions import solve_linear_program
from functions import construct_result_dataframe,Save_errors
import os

In [95]:

chemin_fichier = os.path.join('.', 'data', 'recipe_optimization_data.xlsm')

# Lire les fichiers Excel
df_table, df_MP, df_contraints, erreurs = read_and_check_input(chemin_fichier)

df_contraints_element, df_contraints_qualite, df_MP_dispo, df_MP_indispo = Separate_data(df_table, df_MP, df_contraints)

# Suppression des matières premières indisponibles
df_table = nettoyer_dataframe(df_table, df_MP_indispo)

# Construction de la matrice A et du vecteur C
A, C = construire_tableau(df_table, df_MP_dispo)

# Initialisation des listes pour les contraintes
constraints = {'A_eq': {},'b_eq': {},'A_sup': {},'b_sup': {} }

# Mettre  les contraintes concernant les éléments
constraints = format_constraints_elements(df_contraints_element, A,constraints)

# Mettre  les contraintes concernant la qualité
constraints = format_constraints_qualite(df_contraints_qualite, A,constraints)

# Mettre les contraintes concernant les matières premières disponibles
constraints, bounds = format_constraints_MP(df_MP_dispo, constraints)

# Résoudre le problème d'optimisation linéaire
method = 'simplex' #'interior-point' 'simplex'
res, ce_result, ci_result = solve_linear_program(C, constraints,bounds,method)
conforme = all(val == 1 for val in ce_result) and  all(val == 1 for val in ci_result)    
print(method, '\n')
print(ce_result, ci_result)

simplex 

[1, 0] [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


C:\Users\congoj\AppData\Local\Temp\ipykernel_9320\2726524442.py:12: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res = linprog(C, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds,method = method, options={"presolve": False}) #False


In [ ]:
import pandas as pd

def multiply_component_values(df, components, coefficient):
    """
    Multiplie les valeurs des composants spécifiés dans le DataFrame par un coefficient donné.
    
    Args:
        df (DataFrame): Le DataFrame contenant les données des composants.
        components (list): La liste des composants à multiplier.
        coefficient (float): Le coefficient de multiplication.
    
    Returns:
        DataFrame: Le DataFrame df avec les valeurs des composants multipliées.
    """
    # Créer une copie du DataFrame pour éviter de modifier l'original
    df_copy = df.copy()
    
    # Sélectionner les lignes correspondant aux composants spécifiés et multiplier les valeurs par le coefficient
    df_copy.loc[df_copy['Composant'].isin(components), df_copy.columns[1:]] *= coefficient
    
    return df_copy


In [10]:
# Créer un dictionnaire vide pour stocker les informations des composants
composants_dict = {}
Composant_max = {}
Composant_min = {}
Composant_visee = {}
# Parcourir chaque ligne du DataFrame df_contraints_element
for index, row in df_contraints_element.iterrows():
    # Extraire le composant de la ligne actuelle
    composant = row['Composant']
    # Créer une liste contenant les valeurs 'Valeur Min par four', 'Valeur visée' et 'Valeur Max par four' pour le composant actuel
    valeurs = [row['Valeur Min par four'], row['Valeur visée'], row['Valeur Max par four']]
    # Ajouter cette liste au dictionnaire des composants avec le composant comme clé
    composants_dict[composant] = valeurs

    Composant_min[composant] = row['Valeur Min par four']
    Composant_visee[composant] = row['Valeur visée']
    Composant_max[composant] = row['Valeur Max par four']
    # Composant_max[composant + '_max'] = row['Valeur Max par four']
# Afficher le dictionnaire des composants
print(len(composants_dict))
print(Composant_max)

18
{'C': 3.65, 'Si': 2.2, 'Mn': 0.25, 'Cu': 0.07, 'Cr': 0.05, 'P': 0.03, 'Ni': 0.015, 'Mo': 0.006, 'Sn': 0.015, 'Sb': 0.005, 'Al': 0.018, 'S': 0.03, 'Mg': 0.05, 'Pb': 0.03, 'Ti': 0.015, 'As': 0.005, 'Bi': 0.0015, 'V': 0.005}


In [7]:
Composant_min = df_contraints_element.set_index('Composant')['Valeur Min par four'].to_dict()
Composant_visee = df_contraints_element.set_index('Composant')['Valeur visée'].to_dict()
Composant_max = df_contraints_element.set_index('Composant')['Valeur Max par four'].to_dict()

Composant_Impurete = df_contraints_qualite.set_index('Composant')['Impurété'].to_dict()
Composant_ONO = df_contraints_qualite.set_index('Composant')['ONO'].to_dict()


Impurete_dict = {'C': 0,'Si': 0,'Mn': 0.44,'Cu': 4.90,'Cr': 0.37,'P': 5.60,'Ni': 0.37,
                   'Mo': 7.90,'Sn': 39.00,'Sb': 0,'Al': 0,'S': 0,'Mg': 0,'Pb': 0,'Ti': 4.40,
                   'As': 0,'Bi': 0,'V': 0}

ONO_dict = {'C': 0,'Si': 0,'Mn': 0,'Cu': 1,'Cr': 1,'P': 0,'Ni':1,'Mo': 0,'Sn': 1,'Sb': 1,
            'Al': 1,'S': 0,'Mg': 0,'Pb': 1,'Ti': 1,'As': 1,'Bi': 1,'V': 1}

import numpy as np

def calcul_produit_scalaire(Qualite_dict, Composant):
    """
    Calcule le produit scalaire entre deux dictionnaires si toutes les valeurs dans Composant sont non NaN.

    Args:
        Qualite_dict (dict): Dictionnaire contenant les valeurs ONO.
        Composant (dict): Dictionnaire contenant les valeurs maximales par composant.

    Returns:
        float: Le produit scalaire des deux dictionnaires si aucune valeur n'est NaN, sinon None.
    """
    produit_scalaire = 0
    for key, value in Qualite_dict.items():
        if np.isnan(Composant.get(key)):
            if value != 0:
                return None
        else:
            produit_scalaire += value * Composant[key]
    return produit_scalaire



ps_ONO_max = calcul_produit_scalaire(ONO_dict, Composant_max)
ps_ONO_min = calcul_produit_scalaire(ONO_dict, Composant_min)
ps_Impurete_max = calcul_produit_scalaire(Impurete_dict, Composant_max)
ps_Impurete_min = calcul_produit_scalaire(Impurete_dict, Composant_min)


Max_ONO = df_contraints_qualite.loc[df_contraints_qualite['Composant'] == 'Valeur Max par four', 'ONO'].item()
Min_ONO = df_contraints_qualite.loc[df_contraints_qualite['Composant'] == 'Valeur Min par four', 'ONO'].item()
Max_Impurete = df_contraints_qualite.loc[df_contraints_qualite['Composant'] == 'Valeur Max par four', 'Impurété'].item()
Min_Impurete = df_contraints_qualite.loc[df_contraints_qualite['Composant'] == 'Valeur Min par four', 'Impurété'].item()


if ps_ONO_max :
    Max_ONO = ps_ONO_max
    if Min_ONO > ps_ONO_max :
        print(" Erreur : La Valeur Min par four de ONO est trop grande")


if ps_ONO_min :
    Min_ONO = ps_ONO_min
    if Max_ONO < ps_ONO_min :
        print(" Erreur : La Valeur Max par four de ONO est trop grande")


if ps_Impurete_max :
    Max_Impurete = ps_Impurete_max
    if Min_Impurete > ps_Impurete_max :
        print(" Erreur : La Valeur Min par four de Impurete est trop grande")

if ps_Impurete_min :
    Min_Impurete = ps_Impurete_min
    if Max_Impurete < ps_Impurete_min :
        print(" Erreur : La Valeur Max par four de Impurete est trop grande")
